In [13]:
import pandas as pd

In [14]:
df_trade_history = pd.read_csv("../DATA/SILVER/TradeHistory/TradeHistory_data.csv")

# Convert 'datetime' to datetime type
df_trade_history["datetime"] = pd.to_datetime(df_trade_history["datetime"])
df_trade_history[df_trade_history["stock_name"].str.contains("NIFTY-PE-24650-18JUL2024")]

,datetime,exchange,segment,symbol,stock_name,side,amount,quantity,price,expiry_date
149,2024-07-15 13:23:50,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,BUY,5762.50,50.0,115.25,2024-07-18
150,2024-07-15 14:18:31,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,SELL,5787.50,50.0,115.75,2024-07-18
153,2024-07-16 13:57:37,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,BUY,1941.25,25.0,77.65,2024-07-18
154,2024-07-16 13:57:37,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,BUY,1942.50,25.0,77.70,2024-07-18
155,2024-07-18 09:47:40,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,SELL,3512.50,50.0,70.25,2024-07-18
163,2024-07-18 14:25:45,FON,FO,NaN,NIFTY-PE-24650-18JUL2024,BUY,300.00,100.0,3.00,2024-07-18


In [15]:
# Function to calculate fiscal year in "YYYY-YYYY" format
def get_fiscal_year_range(date):
    start_month = 4
    if date.month >= start_month:
        return f"{date.year}-{date.year + 1}"
    else:
        return f"{date.year - 1}-{date.year}"

In [16]:
df_tax = pd.read_csv("../DATA/BRONZE/TaxReport/TaxReport_data.csv")

# Apply the function to add a new column for fiscal year
df_tax["fy"] = pd.to_datetime(df_tax["close_datetime"]).apply(
    get_fiscal_year_range
)
df_tax["close_datetime"] = pd.to_datetime(
    pd.to_datetime(df_tax["close_datetime"]).dt.date
)
# df_tax = df_tax[df_tax["stock_name"].str.contains("NIFTY-PE-24650-18JUL2024")]
df_tax

,segment,stock_name,quantity,open_datetime,open_price,open_amount,close_datetime,close_price,close_amount,days,pnl_amount,fy
0,EQ,BHAGERIA,10.0,2020-05-05,116.55,1165.50,2020-06-15,113.00,1130.00,41,-35.50,2020-2021
1,EQ,BHAGERIA,5.0,2020-05-19,100.00,500.00,2020-06-15,113.00,565.00,27,65.00,2020-2021
2,EQ,BPCL,43.0,2021-08-06,457.00,19651.00,2022-08-10,328.85,14140.55,369,-5510.45,2022-2023
3,EQ,GOLDBEES,2.0,2021-02-09,41.82,83.64,2021-06-04,42.03,84.06,115,0.42,2021-2022
4,EQ,HERANBA,12.0,2021-03-04,627.00,7524.00,2021-03-05,820.00,9840.00,1,2316.00,2020-2021
...,...,...,...,...,...,...,...,...,...,...,...,...
113,FO,NIFTY-PE-24900-01AUG2024,25.0,2024-07-30,101.95,2548.75,2024-07-30,102.75,2568.75,0,20.00,2024-2025
114,FO,NIFTY-PE-24900-01AUG2024,25.0,2024-07-30,101.95,2548.75,2024-07-30,102.75,2568.75,0,20.00,2024-2025
115,FO,NIFTY-PE-24950-01AUG2024,25.0,2024-07-31,98.85,2471.25,2024-07-31,85.45,2136.25,0,-335.00,2024-2025
116,FO,NIFTY-PE-24950-01AUG2024,125.0,2024-08-01,7.00,875.00,2024-08-01,0.05,6.25,0,-868.75,2024-2025


In [17]:


# df_tax["stock_name"] = df_tax["stock_name"].apply(lambda a: "-".join(a.split("-")[1:-1]) if len(a.split("-")) > 1 else a)
df_tax = (
    df_tax.groupby(
        [
            "close_datetime",
            "segment",
            "stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(
        by=[
            "segment",
            "stock_name",
        ]
    )
    .reset_index()
)
print(df_tax["pnl_amount"].sum())
df_tax

10519.95


,close_datetime,segment,stock_name,pnl_amount
0,2020-06-15,EQ,BHAGERIA,29.50
1,2022-08-10,EQ,BPCL,-5510.45
2,2021-06-04,EQ,GOLDBEES,0.42
3,2021-03-05,EQ,HERANBA,2316.00
4,2021-03-09,EQ,HERANBA,1628.00
...,...,...,...,...
59,2024-07-29,FO,NIFTY-PE-24900-01AUG2024,-3085.00
60,2024-07-30,FO,NIFTY-PE-24900-01AUG2024,40.00
61,2024-07-31,FO,NIFTY-PE-24950-01AUG2024,-335.00
62,2024-08-01,FO,NIFTY-PE-24950-01AUG2024,-868.75


In [18]:
# filepath of the CSV file
filepath = "../DATA/GOLD/ProfitLoss/ProfitLoss_data.csv"

# Read the CSV file into a DataFrame
df_pnl = pd.read_csv(filepath)

df_pnl["close_datetime"] = pd.to_datetime(
    pd.to_datetime(df_pnl["close_datetime"]).dt.date
)
# df_pnl = df_pnl[df_pnl["stock_name"].str.contains("NIFTY-PE-24650-18JUL2024")]
df_pnl

,exchange,segment,stock_name,position,quantity,open_datetime,open_side,open_price,open_amount,close_datetime,close_side,close_price,close_amount,pnl_amount,pnl_percentage
0,NSE,EQ,BHAGERIA,LONG,10.0,2020-05-05 11:31:20,BUY,116.55,1165.50,2020-06-15,SELL,113.00,1130.00,-35.50,-3.05
1,NSE,EQ,BHAGERIA,LONG,5.0,2020-05-19 14:44:06,BUY,100.00,500.00,2020-06-15,SELL,113.00,565.00,65.00,13.00
2,NSE,EQ,BPCL,LONG,43.0,2021-08-06 11:42:14,BUY,457.00,19651.00,2022-08-10,SELL,328.85,14140.55,-5510.45,-28.04
3,NSE,EQ,GOLDBEES,LONG,2.0,2021-02-09 14:25:00,BUY,41.82,83.64,2021-06-04,SELL,42.03,84.06,0.42,0.50
4,NSE,EQ,HERANBA,LONG,12.0,2021-03-04 00:00:00,BUY,627.00,7524.00,2021-03-05,SELL,820.00,9840.00,2316.00,30.78
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,FON,FO,NIFTY-PE-24900-01AUG2024,LONG,25.0,2024-07-30 15:09:20,BUY,101.95,2548.75,2024-07-30,SELL,102.75,2568.75,20.00,0.78
116,FON,FO,NIFTY-PE-24900-01AUG2024,LONG,25.0,2024-07-30 15:09:20,BUY,101.95,2548.75,2024-07-30,SELL,102.75,2568.75,20.00,0.78
117,FON,FO,NIFTY-PE-24950-01AUG2024,LONG,25.0,2024-07-31 11:19:46,BUY,98.85,2471.25,2024-07-31,SELL,85.45,2136.25,-335.00,-13.56
118,FON,FO,NIFTY-PE-24950-01AUG2024,LONG,125.0,2024-08-01 14:34:46,BUY,7.00,875.00,2024-08-01,SELL,0.05,6.25,-868.75,-99.29


In [19]:
# "EXPIRED" sotck are comming as postive/

In [20]:

# df_pnl["stock_name"] = df_pnl["stock_name"].apply(lambda a: "-".join(a.split("-")[1:-1]) if len(a.split("-")) > 1 else a)
df_pnl = (
    df_pnl.groupby(
        [
            "close_datetime",
            "segment",
            "stock_name",
        ]
    )
    .agg({"pnl_amount": "sum"})
    .sort_values(
        by=[
            "segment",
            "stock_name",
        ]
    )
    .reset_index()
)
print(df_pnl["pnl_amount"].sum())
df_pnl

10519.95


,close_datetime,segment,stock_name,pnl_amount
0,2020-06-15,EQ,BHAGERIA,29.50
1,2022-08-10,EQ,BPCL,-5510.45
2,2021-06-04,EQ,GOLDBEES,0.42
3,2021-03-05,EQ,HERANBA,2316.00
4,2021-03-09,EQ,HERANBA,1628.00
...,...,...,...,...
59,2024-07-29,FO,NIFTY-PE-24900-01AUG2024,-3085.00
60,2024-07-30,FO,NIFTY-PE-24900-01AUG2024,40.00
61,2024-07-31,FO,NIFTY-PE-24950-01AUG2024,-335.00
62,2024-08-01,FO,NIFTY-PE-24950-01AUG2024,-868.75


In [21]:
# Merge DataFrames on 'stock_name'
df_merged = pd.merge(
    df_pnl,
    df_tax,
    on=[
        "close_datetime",
        "segment",
        "stock_name",
    ],
    suffixes=("_pnl", "_tax"),
    how="outer",
)

# Compare pnl_amount from both DataFrames
df_merged["pnl_match"] = (
    df_merged["pnl_amount_pnl"] == df_merged["pnl_amount_tax"]
)
df_merged = df_merged[df_merged["pnl_match"] == False]
# Remove duplicate stock names, if necessary
# df_merged = df_merged.drop_duplicates(subset=['stock_name'])
df_merged.sort_values(by=["pnl_match", "stock_name"]).fillna(0).reset_index(
    drop=True
)

,close_datetime,segment,stock_name,pnl_amount_pnl,pnl_amount_tax,pnl_match
